<a href="https://colab.research.google.com/github/Imran2205/LogicRAG/blob/master/inference/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/Imran2205/LogicRAG

Cloning into 'LogicRAG'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 709 (delta 15), reused 28 (delta 7), pack-reused 663 (from 1)
Receiving objects: 100% (709/709), 35.81 MiB | 16.49 MiB/s, done.
Resolving deltas: 100% (220/220), done.


In [2]:
! gdown 14vRnzCDD9rw63Os3w-Ii4ZF_352y__km

Downloading...
From (original): https://drive.google.com/uc?id=14vRnzCDD9rw63Os3w-Ii4ZF_352y__km
From (redirected): https://drive.google.com/uc?id=14vRnzCDD9rw63Os3w-Ii4ZF_352y__km&confirm=t&uuid=7693222e-61b3-4dc8-ab46-2252e25664e4
To: /content/LogicRAG_Data.zip
100% 2.26G/2.26G [00:11<00:00, 201MB/s]


In [3]:
! unzip LogicRAG_Data.zip

Archive:  LogicRAG_Data.zip
   creating: LogicRAG_Data/
   creating: LogicRAG_Data/model_weights/
  inflating: LogicRAG_Data/.DS_Store  
  inflating: __MACOSX/LogicRAG_Data/._.DS_Store  
   creating: LogicRAG_Data/images/
   creating: LogicRAG_Data/precomputed_knowledge_base/
   creating: LogicRAG_Data/tracker_trajectories/
   creating: LogicRAG_Data/questions/
   creating: LogicRAG_Data/model_weights/PixelFormer/
  inflating: LogicRAG_Data/model_weights/.DS_Store  
  inflating: __MACOSX/LogicRAG_Data/model_weights/._.DS_Store  
   creating: LogicRAG_Data/model_weights/Mask2Former/
   creating: LogicRAG_Data/model_weights/ColorDetector/
   creating: LogicRAG_Data/model_weights/TypeDetector/
   creating: LogicRAG_Data/model_weights/CoTracker3/
  inflating: LogicRAG_Data/images/.DS_Store  
  inflating: __MACOSX/LogicRAG_Data/images/._.DS_Store  
   creating: LogicRAG_Data/images/0003/
   creating: LogicRAG_Data/images/0004/
   creating: LogicRAG_Data/images/0005/
   creating: LogicRAG_Da

In [4]:
! pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 5.4 MB/s eta 0:00:00


In [5]:
import os
import sys
import logging
import pandas as pd

current_directory = os.getcwd()
project_root = os.path.abspath('/content/LogicRAG')
sys.path.append(project_root)

from LogicRAG.kb_framework.kb_inference.inference_in_kb import lrag_inference
from LogicRAG.baselines.get_gpt4_response import get_answers_gpt4
from LogicRAG.baselines.get_claude_response import get_answers_claude3

from LogicRAG.kb_framework.kb_inference.utils.eval_utils import get_scores

In [6]:
! mkdir -p log

In [7]:
que_csv = "/content/LogicRAG_Data/questions/kitti_que.csv"
fol_trans_csv = "/content/LogicRAG_Data/questions/translated_fol_queries.csv"
kb_dir = "/content/LogicRAG_Data/precomputed_knowledge_base/kb_out_kitti"
tracker_dir = "/content/LogicRAG_Data/tracker_trajectories/track_out_kitti"
output = "./kitti_que_and_model_responses.csv"
frames_dir = "/content/LogicRAG_Data/images"
log_dir = "./log"

GPT_MODEL_NAME = "gpt-4o"  # gpt-4o-2024-08-06
CLAUDE_MODEL_NAME = "claude-3-5-sonnet-20240620"

os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_api_key"
os.environ["OPENAI_API"] = "your_openai_api_key"

logging.basicConfig(level=logging.INFO)
base_logger = logging.getLogger(__name__)

In [8]:
all_answers, all_annotations = lrag_inference(
      que_csv, fol_trans_csv,
      kb_dir, tracker_dir,
      logger=base_logger
)

df = pd.read_csv(que_csv)
df['LogicRAG'] = all_answers
df.to_csv(output, index=False)

In [ ]:
get_answers_claude3(
    output,
    frames_dir,
    output,
    log_dir,
    CLAUDE_MODEL_NAME
)

In [ ]:
get_answers_gpt4(
    output,
    frames_dir,
    output,
    log_dir,
    GPT_MODEL_NAME
)

In [ ]:
df = pd.read_csv(output)

In [ ]:
df.head(5)

,Video,Frame,Questions,Human,LogicRAG,claude-3-5-sonnet-20240620,gpt-4o
0,0,000060-000069,Can you spot a pedestrian at the right in the ...,1,1,1,1
1,0,000060-000069,Is there a White Car on the right of the frame?,1,1,1,0
2,0,000060-000069,Does the distance between the pedestrian and t...,1,1,1,1
3,0,000060-000069,Is there a Black Car on the left the scene?,1,1,1,1
4,0,000060-000069,Does the Black Car disappear from the scene?,1,1,0,0


In [ ]:
acc_lrag, f1_lrag, _, _ = get_scores(
    df['LogicRAG'], df['Human']
)
acc_claude, f1_claude, _, _ = get_scores(
    df[CLAUDE_MODEL_NAME], df['Human']
)
acc_gpt, f1_gpt, _, _ = get_scores(
    df[GPT_MODEL_NAME], df['Human']
)

print(f"LogicRAG Accuracy: {acc_lrag:.2f}, F1: {f1_lrag:.2f}")
print(f"Claude 3 Accuracy: {acc_claude:.2f}, F1: {f1_claude:.2f}")
print(f"GPT-4V   Accuracy: {acc_gpt:.2f}, F1: {f1_gpt:.2f}")

LogicRAG Accuracy: 0.94, F1: 0.97
Claude 3 Accuracy: 0.75, F1: 0.85
GPT-4V   Accuracy: 0.68, F1: 0.79
